## 0.1 imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
from tqdm import tqdm
from pathlib import Path
from typing import List

In [2]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, MinMaxScaler, FunctionTransformer
from sklearn.pipeline import Pipeline, make_pipeline, make_union
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import make_column_transformer, ColumnTransformer

from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold, train_test_split

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset, TensorDataset

# 1. Load the raw data

#### **Collected data**

#### 📌 Tempelhofer Feld, Berlin, Germany

<iframe src="https://www.google.com/maps/embed?pb=!1m18!1m12!1m3!1d26842.24808091901!2d13.400488957085193!3d52.477045271245125!2m3!1f0!2f0!3f0!3m2!1i1024!2i768!4f13.1!3m3!1m2!1s0x47a84fe8f7d899eb%3A0x88898e99acbb718b!2sTempelhofer%20Feld!5e0!3m2!1sen!2sde!4v1736167638260!5m2!1sen!2sde" width="1200" height="450" style="border:0;" allowfullscreen="" loading="lazy" referrerpolicy="no-referrer-when-downgrade"></iframe>


### ☀️ 1. Historical photovoltaic production data
* 42 years (1980 - 2022)
* hourly resolution
* synthetic data derived from measured irradiance data
* ~ 400 000 data points
* source: [Renewables.ninja](https://www.renewables.ninja/about "Renewables.ninja")

### 🌡️ 2. Historical weather forecast data
* 6 1/2 years (Oct 2017 - Mar 2024)
* 4 model recalculation cycles available for each day (updated at: 00:00 UTC, 06:00 UTC, 12:00 UTC, 18:00 UTC)
* Each 16 days forecast in hourly resolution
* ~ 3.3 M rows - 24 Features
* source: [OpenWeather](https://openweathermap.org/api/history-forecast-bulk "History-forecast-bulk") (40 €)


In [4]:
pv_data = pd.read_csv('./data/1980-2022_pv.csv')
forecast_data = pd.read_csv('./data/openweather_history_bulk_forecast_tempelhof.csv')

# 2. Check the data

## 2.1 Photovolatic data

In [5]:
pv_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376944 entries, 0 to 376943
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0.1        376944 non-null  int64  
 1   Unnamed: 0          376944 non-null  int64  
 2   local_time          376944 non-null  object 
 3   electricity         376944 non-null  float64
 4   irradiance_direct   376944 non-null  float64
 5   irradiance_diffuse  376944 non-null  float64
 6   temperature         376944 non-null  float64
 7   source              376944 non-null  object 
dtypes: float64(4), int64(2), object(2)
memory usage: 23.0+ MB


In [6]:
pv_data.isna().sum()

Unnamed: 0.1          0
Unnamed: 0            0
local_time            0
electricity           0
irradiance_direct     0
irradiance_diffuse    0
temperature           0
source                0
dtype: int64

In [7]:
pv_data.isnull().sum()

Unnamed: 0.1          0
Unnamed: 0            0
local_time            0
electricity           0
irradiance_direct     0
irradiance_diffuse    0
temperature           0
source                0
dtype: int64

In [ ]:
pv_data.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0.1,376944.0,4.382576e+03,2.530581e+03,0.000000e+00,2.191000e+03,4.383000e+03,6.574000e+03,8.783000e+03
Unnamed: 0,376944.0,9.940302e+11,3.917322e+11,3.155328e+11,6.547815e+11,9.940302e+11,1.333279e+12,1.672528e+12
electricity,376944.0,1.345303e-01,2.148500e-01,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e-01,9.000000e-01
irradiance_direct,376944.0,1.068965e-01,2.120717e-01,0.000000e+00,0.000000e+00,0.000000e+00,7.700000e-02,1.016000e+00
irradiance_diffuse,376944.0,5.566084e-02,7.542225e-02,0.000000e+00,0.000000e+00,4.000000e-03,1.020000e-01,3.580000e-01
temperature,376944.0,9.219034e+00,9.013729e+00,-2.745800e+01,2.217000e+00,8.863000e+00,1.591600e+01,3.875600e+01


### 2.1.1 check for time gaps in data

In [9]:
# number of rows with unique values in our data
local_time_series = pd.to_datetime(pv_data['local_time'], utc= True)

unique_timestamps_pv_dataset = local_time_series.nunique()
unique_timestamps_pv_dataset

376944

In [10]:
# number of rows with unique values of a synthetic data with the same range
min_date = local_time_series.min()
max_date = local_time_series.max()
unique_timestamps_pv_dataset_theoretical = pd.date_range(start= min_date, end= max_date, freq=dt.timedelta(hours=1.0)).nunique()
unique_timestamps_pv_dataset_theoretical

376944

In [11]:
try:
    assert unique_timestamps_pv_dataset == unique_timestamps_pv_dataset_theoretical
    print('all good! there are no gaps in your dataset...')
except AssertionError:
    print('there are gaps in your dataset')

all good! there are no gaps in your dataset...


# 3. Preprocess the data

## 3.1 Photovolatic data

In [12]:
pv_data.head()

,Unnamed: 0.1,Unnamed: 0,local_time,electricity,irradiance_direct,irradiance_diffuse,temperature,source
0,0,315532800000,1980-01-01 01:00:00+01:00,0.0,0.0,0.0,-1.296,data/pv_data/1980_pv.csv
1,1,315536400000,1980-01-01 02:00:00+01:00,0.0,0.0,0.0,-1.216,data/pv_data/1980_pv.csv
2,2,315540000000,1980-01-01 03:00:00+01:00,0.0,0.0,0.0,-1.005,data/pv_data/1980_pv.csv
3,3,315543600000,1980-01-01 04:00:00+01:00,0.0,0.0,0.0,-1.063,data/pv_data/1980_pv.csv
4,4,315547200000,1980-01-01 05:00:00+01:00,0.0,0.0,0.0,-1.227,data/pv_data/1980_pv.csv


In [ ]:
def time_encoding(X: pd.DataFrame, column_name: str) -> pd.DataFrame:
    """
    Convert local time into cyclic features to feed significant signal
    in ML / DL algorithm

    Args:
        X (pd.DataFrame): datafrane to transform

    Returns:
        DataFrame with 4 addtional features per time column
    """
    X = X.copy()

    local_time = pd.to_datetime(X.pop(column_name), utc= True)
    timestamp_s = local_time.map(pd.Timestamp.timestamp)


    day = 24*60*60
    year = (365.2425)*day
    X['day_sin'] = np.sin(timestamp_s * (2 * np.pi / day))
    X['day_cos'] = np.cos(timestamp_s * (2 * np.pi / day))
    X['year_sin'] = np.sin(timestamp_s * (2 * np.pi / year))
    X['year_cos'] = np.cos(timestamp_s * (2 * np.pi / year))

    return X.iloc[:,-4:]


time_encoder = FunctionTransformer(time_encoding, kw_args={'column_name': 'local_time'})

pv_transformer = ColumnTransformer(
    [
        ('Time Encoder', time_encoder, ['local_time']),
        ('Passthrough', 'passthrough', ['local_time', 'electricity' ]),
    ],
    remainder='drop'
).set_output(transform='pandas')

preprocess_pv = Pipeline([('Time Features', pv_transformer)])
pv_processed = preprocess_pv.fit_transform(pv_data)

pv_processed.columns = [column.split('__')[1] for column in pv_processed.columns]
pv_processed

In [ ]:
melted_pv_df = pd.melt(pv_processed.iloc[:,:])

plt.figure(figsize=(8, 3))
sns.violinplot(data=melted_pv_df.sample(10_000), x='variable', y='value')

In [ ]:
column_indices = {name:i for i, name in enumerate(pv_processed.columns)}
column_indices

## 3.2 Weather forecast data

In [ ]:
forecast_data.info()

In [ ]:
def compress(df: pd.DataFrame, **kwargs) -> pd.DataFrame:
    """
    Reduces size of dataframe by downcasting numerical columns
    """
    input_size = df.memory_usage(index=True).sum() / 1024
    print("dataframe size: ", round(input_size,2), 'kB')

    in_size = df.memory_usage(index=True).sum()
    for type in ["float", "integer"]:
        l_cols = list(df.select_dtypes(include=type))
        for col in l_cols:
            df[col] = pd.to_numeric(df[col], downcast=type)
    out_size = df.memory_usage(index=True).sum()
    ratio = (1 - round(out_size / in_size, 2)) * 100

    print("optimized size by {} %".format(round(ratio,2)))
    print("new dataframe size: ", round(out_size / 1024,2), " kB")

    return df

def clean_forecast_data(forecast_df: pd.DataFrame) -> pd.DataFrame:
    """
    Initial has 3.3 M entries (everyday: 4 forecasts of 16 days ahead)
    Cleaning it to: - 1 forecast perday (at 12:00)
                    - 48 hours a day
                    - right now hardcoded to match last forecast day with
                     last day of PV data
    """
    df = compress(forecast_df)
    df = df.drop(columns=['lat', 'lon',
                          'forecast dt iso',
                          'slice dt iso'])

    df.rename(columns={'forecast dt unixtime':'utc_time',
                        'slice dt unixtime':'prediction_utc_time'},
                        inplace=True)

    # df['utc_time'] = df['utc_time'].str.replace('+0000 UTC', '')
    # df['prediction_utc_time'] = df['prediction_utc_time'].str.replace('+0000 UTC', '')

    df['utc_time'] = pd.to_datetime(df['utc_time'], unit= 's', utc= True)
    df['prediction_utc_time'] = pd.to_datetime(df['prediction_utc_time'], unit= 's', utc= True)

    # # get only 1 forecast per day
    df = df[df['utc_time'].dt.hour == 12]

    unique_dates = df['utc_time'].unique()

    # reduce to 24h of weather forecast (from 00:00 to 23:00 each day)
    df_revised = []
    for date in unique_dates:
        data = df[(df['utc_time'] == date)].iloc[12:36]
        df_revised.append(data)

    df_revised_ordered = pd.concat(df_revised, ignore_index=True)

    # # hard code the end date to match wiht PV data
    # processed_df = df_revised_ordered[df_revised_ordered['prediction_utc_time'] <= '2022-12-31 23:00:00']

    return df_revised_ordered

forecast_clean = clean_forecast_data(forecast_data)
forecast_clean

In [ ]:
# stick electricity data to the forecast data
pv_data['local_time'] = pd.to_datetime(pv_data['local_time'], utc= True)
merged_forecast_pv = pd.merge(pv_data[['local_time', 'electricity']], forecast_clean, left_on='local_time', right_on= 'prediction_utc_time', how='inner')
merged_forecast_pv.info()

In [ ]:
def time_encoding(X: pd.DataFrame) -> pd.DataFrame:
    """
    Convert local time into cyclic features to feed significant signal
    in ML / DL algorithm

    Args:
    X: datafrane to transform

    Output:
    DataFrame with 4 addtional features per time column
    """
    X = X.copy()

    date_time = pd.to_datetime(X.pop('prediction_utc_time'), utc= True)
    timestamp_s = date_time.map(pd.Timestamp.timestamp)

    day = 24*60*60
    year = (365.2425)*day
    X['forecast_day_sin'] = np.sin(timestamp_s * (2 * np.pi / day))
    X['forecast_day_cos'] = np.cos(timestamp_s * (2 * np.pi / day))
    X['forecast_year_sin'] = np.sin(timestamp_s * (2 * np.pi / year))
    X['forecast_year_cos'] = np.cos(timestamp_s * (2 * np.pi / year))

    return X.iloc[:,-4:]

def wind_encoding(X: pd.DataFrame) -> pd.DataFrame:
    """
    Convert local time into cyclic features to feed significant signal
    in ML / DL algorithm

    Args:
    X: datafrane to transform

    Output:
    DataFrame with 4 addtional features per time column
    """
    X = X.copy()

    # Process wind fratures
    wind_speed = X.pop('wind_speed')

    # Convert to radians.
    wind_rad = X.pop('wind_deg')*np.pi / 180

    # Calculate the wind x and y components
    X['Wx'] = wind_speed*np.cos(wind_rad)
    X['Wy'] = wind_speed*np.sin(wind_rad)

    # Standardize the components
    X['Wx'] = (X['Wx'] - X['Wx'].mean())/X['Wx'].std()
    X['Wy'] = (X['Wy'] - X['Wy'].mean())/X['Wy'].std()

    return X.iloc[:,-2:]

std_features = ['temperature', 'dew_point', 'pressure', 'ground_pressure',
                'humidity',]
minmax_features = ['clouds', 'rain', 'snow', 'ice', 'fr_rain', 'convective',
                   'snow_depth', 'accumulated', 'hours', 'rate', 'probability']

time_encoder = FunctionTransformer(time_encoding)
wind_encoder = FunctionTransformer(wind_encoding)

forecast_transformer = ColumnTransformer(
    [
        ('Time Encoder', time_encoder, ['prediction_utc_time']),
        ('Wind Encoder', wind_encoder, ['wind_speed', 'wind_deg']),
        ('Std', StandardScaler(), std_features),
        ('MinMax', MinMaxScaler(), minmax_features),
        # ('Drop', 'drop', ['local_time', 'utc_time', 'prediction_utc_time']),
        ('Passthrough', 'passthrough', ['utc_time', 'prediction_utc_time']),
    ],
    remainder='passthrough'
).set_output(transform='pandas')


forecast_processed = forecast_transformer.fit_transform(merged_forecast_pv)

forecast_processed.columns = [column.split('__')[1] for column in forecast_processed.columns]
display(forecast_processed)
forecast_transformer


In [ ]:
plt.figure(figsize=(16, 5))

plt.subplot(311)
melted_forecast_df = pd.melt(forecast_processed.iloc[:, :8])
sns.violinplot(data= melted_forecast_df.sample(frac=0.25), x='variable', y='value')

plt.subplot(312)
melted_forecast_df = pd.melt(forecast_processed.iloc[:, 8:16])
sns.violinplot(data= melted_forecast_df.sample(frac=0.25), x='variable', y='value')
plt.tight_layout()

plt.subplot(313)
melted_forecast_df = pd.melt(forecast_processed.iloc[:, 16:])
sns.violinplot(data= melted_forecast_df.sample(frac=0.25), x='variable', y='value')
plt.tight_layout()

In [ ]:
forecast_processed.info()

# 4. Sequencing

## 4.1 Photovoltaic Data

The goal here is to train the model on a high number of sequences. Time series data have that in particular that 

In [ ]:
#### Parameters ################
#################################

INPUT_WIDTH = 48
LABEL_WIDTH = 24
SHIFT = 36

class SequenceGenerator():
    """

    """
    def __init__(self, input_width: int, label_width: int, shift: int, number_sequences: int,
               train_df: pd.DataFrame, val_df: pd.DataFrame =None, test_df: pd.DataFrame =None,
               label_columns: List[str] =None):

        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in
                                            enumerate(label_columns)}
        self.column_indices = {name: i for i, name in
                            enumerate(train_df.columns)}

        # Work out the window parameters.
        self.number_of_sequences = number_sequences
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, self.total_window_size)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Number of sequences: {self.number_of_sequences}',
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])

    def split_window(self, features):
        inputs = features[self.input_slice, :]
        labels = features[self.labels_slice, :]
        if self.label_columns is not None:
            labels = np.stack(
                [labels[:, self.column_indices[name]] for name in self.label_columns],
                axis=-1)

        return inputs, labels


    def make_dataset(self, data: pd.DataFrame):
        "Create a dataset of x sequences of features and labels"
        data = np.array(data)
        last_full_sequence_start = len(data)- self.total_window_size
        inputs, labels = [], []

        for n in tqdm(range(self.number_of_sequences)):
            random_start = np.random.randint(0, last_full_sequence_start)
            input, label = self.split_window(data[random_start:])

            inputs.append(torch.tensor(input, dtype=torch.float32))
            labels.append(torch.tensor(label, dtype=torch.float32))

        inputs = torch.cat(inputs).view(self.number_of_sequences, self.input_width, -1)
        labels = torch.cat(labels).view(self.number_of_sequences, self.label_width, -1)
        return TensorDataset(inputs, labels)

    @property
    def train(self):
        return self.make_dataset(self.train_df)

    @property
    def val(self):
        return self.make_dataset(self.val_df)

    @property
    def test(self):
        return self.make_dataset(self.test_df)

    @property
    def example(self):
        """Get and cache an example batch of `inputs, labels` for plotting."""
        result = getattr(self, '_example', None)
        if result is None:
            # No example batch was found, so get one from the `.train` dataset
            result = next(iter(self.train))
            # And cache it for next time
            self._example = result
        return result



seq_pv = SequenceGenerator(input_width= INPUT_WIDTH,
                              label_width= LABEL_WIDTH,
                              shift= SHIFT,
                              number_sequences= 10_000,
                              train_df= pv_processed,
                              label_columns= ['electricity'])

seq_pv

## 4.2 Forecast Data

In [ ]:
class SequenceForecastDataset(Dataset):
    def __init__(self, df: pd.DataFrame, number_days_forecast: int = 1,
                 label_columns: List[str] = None):
        self.df = df.copy().astype('float32')
        self.forecast_hours = number_days_forecast * 24
        self.number_of_sequences = len(self.df) // self.forecast_hours

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in
                                            enumerate(label_columns)}
        self.column_indices = {name: i for i, name in
                            enumerate(self.df.columns)}
        self.feature_columns = [col for col in self.df.columns if col not in self.label_columns]

    def __len__(self):
        return self.number_of_sequences

    def __getitem__(self, index):
        inputs = self.df[self.feature_columns].values
        if self.label_columns is not None:
            labels = self.df[[name for name in self.label_columns]].values
        else:
            labels = self.df['electricity'].values
        return inputs.reshape(self.number_of_sequences, self.forecast_hours, -1)[index],\
            labels.reshape(self.number_of_sequences, self.forecast_hours, -1)[index]

    def __repr__(self):
        return '\n'.join([
            f'Number of sequences: {self.number_of_sequences}',
            f'Label column name(s): {self.label_columns}'])

    @property
    def example(self):
        """Get and cache an example batch of `inputs, labels` for plotting."""
        result = getattr(self, '_example', None)
        if result is None:
            # No example batch was found, so get a random one from the dataset
            random_idx = np.random.randint(0, self.number_of_sequences)
            result = self[random_idx]
            # And cache it for next time
            self._example = result
        return result


forecast_processed = forecast_processed.select_dtypes(include=np.number)
seq_forecast = SequenceForecastDataset(forecast_processed, label_columns=['electricity', 'probability'])
seq_forecast


# 5. Deep Learning

## 5.1 Split the data

In [ ]:
# Split the PV data

column_indices = {name: i for i, name in enumerate(pv_processed.columns)}

n = len(pv_processed)
train_df_pv = pv_processed[0:int(n*0.7)]
val_df_pv = pv_processed[int(n*0.7):int(n*0.9)]
test_df_pv = pv_processed[int(n*0.9):]

In [ ]:
# Split the Forecast data
# Careful: split must be done for 24h data

column_indices = {name: i for i, name in enumerate(forecast_processed.columns)}

n = len(forecast_processed)
train_df_forecast = forecast_processed[0:int(n*0.7 / 24) * 24]
val_df_forecast = forecast_processed[int(n*0.7 / 24) * 24:int(n*0.9 / 24) * 24]
test_df_forecast = forecast_processed[int(n*0.9 / 24) * 24:]

## 5.2 Baseline metrics

### 5.2.1 Compute regression metrics function

In [ ]:
def compute_regression_metrics(model, dataloader):
    y_preds = []
    labels = []

    with torch.no_grad():
        for inputs, targets in dataloader:
            outputs = model(inputs)
            y_preds.append(outputs.cpu())
            labels.append(targets.cpu())

    y_preds = torch.cat(y_preds)
    labels = torch.cat(labels)

    # Compute metrics
    mse = torch.mean((y_preds - labels) ** 2)
    rmse = mse ** 0.5
    mae = torch.mean(abs(y_preds - labels))
    r2 = 1 - torch.sum((y_preds - labels) ** 2) / torch.sum(y_preds - torch.mean(labels) ** 2)

    return {"mse": mse, "rmse": rmse, "mae": mae, "r2": r2}

### 5.2.2 Photovoltaic data

In [ ]:
sequences_pv = SequenceGenerator(input_width= INPUT_WIDTH,
                              label_width= LABEL_WIDTH,
                              shift= SHIFT,
                              number_sequences= 10_000,
                              train_df= train_df_pv,
                              val_df= val_df_pv,
                              test_df= test_df_pv,
                              label_columns= ['electricity'])

train_dataset_pv = sequences_pv.train
val_dataset_pv = sequences_pv.val
test_dataset_pv = sequences_pv.test

In [ ]:
test_loader_pv = DataLoader(test_dataset_pv, batch_size= 32, shuffle= True)

In [ ]:
class BaselinePV(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        x = x[:, 12:-12, 4]
        return x.view(len(x), -1, 1)


In [ ]:
baseline_pv = compute_regression_metrics(BaselinePV(), test_loader_pv)

print("Baseline Metrics:", baseline_pv)

### 5.2.3 Forecast data 

In [ ]:
# stick day_ahead and day_before electricity data to the forecast data
pv_data['local_time'] = pd.to_datetime(pv_data['local_time'], utc= True)
merged_forecast_pv = pd.merge(pv_data[['local_time', 'electricity']], forecast_clean, left_on='local_time', right_on= 'prediction_utc_time', how='inner')
merged_forecast_pv = merged_forecast_pv.rename(columns= {'electricity': 'electricity_day_ahead'})

merged_forecast_pv['baseline_day_utc_time'] = merged_forecast_pv['prediction_utc_time'] - dt.timedelta(days= 2)
merged_forecast_pv = pd.merge(pv_data[['local_time', 'electricity']], merged_forecast_pv, left_on='local_time', right_on= 'baseline_day_utc_time', how='inner')
merged_forecast_pv = merged_forecast_pv.rename(columns= {'electricity': 'electricity_day_before'})

time_cols_mask = [col for col in merged_forecast_pv.columns if 'time' not in col]

baseline_forecast_df = merged_forecast_pv[time_cols_mask]
baseline_forecast_dataset = SequenceForecastDataset(df= baseline_forecast_df,
                                                label_columns= ['electricity_day_ahead'])
display(baseline_forecast_dataset.df), baseline_forecast_dataset

In [ ]:
# Split the Forecast data
# Careful: split must be done for 24h data

column_indices = {name: i for i, name in enumerate(baseline_forecast_df.columns)}

n = len(baseline_forecast_df)
train_baseline_df_forecast = baseline_forecast_df[0:int(n*0.7 / 24) * 24]
val_baseline_df_forecast = baseline_forecast_df[int(n*0.7 / 24) * 24:int(n*0.9 / 24) * 24]
test_df_baseline_forecast = baseline_forecast_df[int(n*0.9 / 24) * 24:]

In [ ]:
test_baseline_dataset = SequenceForecastDataset(df= test_df_baseline_forecast,
                                            label_columns= ['electricity_day_ahead'])

test_baseline_loader_forecast = DataLoader(test_baseline_dataset, batch_size= 32, shuffle= True)
test_baseline_dataset.column_indices

In [ ]:
class BaselineForecast(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        x = x[:, :, 0]
        return x.view(len(x), -1, 1)

baseline_forecast = compute_regression_metrics(BaselineForecast(), test_baseline_loader_forecast)
print("Baseline Metrics:", baseline_forecast)

## 5.3 Prepare the datasets and dataloaders

In [ ]:
# Split the PV data

column_indices = {name: i for i, name in enumerate(pv_processed.columns)}

n = len(pv_processed)
train_df_pv = pv_processed[0:int(n*0.7)]
val_df_pv = pv_processed[int(n*0.7):int(n*0.9)]
test_df_pv = pv_processed[int(n*0.9):]

sequences_pv = SequenceGenerator(input_width= INPUT_WIDTH,
                              label_width= LABEL_WIDTH,
                              shift= SHIFT,
                              number_sequences= 10_000,
                              train_df= train_df_pv,
                              val_df= val_df_pv,
                              test_df= test_df_pv,
                              label_columns= ['electricity'])

train_dataset_pv = sequences_pv.train
val_dataset_pv = sequences_pv.val
test_dataset_pv = sequences_pv.test

train_loader_pv = DataLoader(train_dataset_pv, batch_size= 32, shuffle= True)
val_loader_pv = DataLoader(val_dataset_pv, batch_size= 32, shuffle= True)
test_loader_pv = DataLoader(test_dataset_pv, batch_size= 32, shuffle= True)

In [ ]:
train_dataset_forecast = SequenceForecastDataset(df= train_df_forecast, label_columns=['electricity'])
val_dataset_forecast = SequenceForecastDataset(df= val_df_forecast, label_columns=['electricity'])
test_dataset_forecast = SequenceForecastDataset(df= test_df_forecast, label_columns=['electricity'])

train_loader_forecast = DataLoader(train_dataset_forecast, batch_size= 16, shuffle= True)
val_loader_forecast = DataLoader(val_dataset_forecast, batch_size= 16, shuffle= True)
test_loader_forecast = DataLoader(test_dataset_forecast, batch_size= 16, shuffle= True)

## 5.4 Simple LSTM models

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, p: int):
        super(LSTMModel, self).__init__()
        self.n_features = p
        self.lstm = nn.LSTM(p, 24, batch_first= True)
        self.linear = nn.Linear(24,24)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = F.tanh(x)
        x = self.linear(x[:,-1,:])
        return x.view(len(x), -1, 1)

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, p: int):
        super(RNNModel, self).__init__()
        self.n_features = p
        self.lstm = nn.RNN(p, 24, batch_first= True)
        self.linear = nn.Linear(24,24)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = F.tanh(x)
        x = self.linear(x[:,-1,:])
        return x.view(len(x), -1, 1)

In [ ]:
train_loader_forecast.__len__(), val_loader_forecast.__len__(), test_loader_forecast.__len__()

In [ ]:
############# Data ##############
dataset = 'forecast'  # ['pv', 'forecast']

########## Parameters #########
batch_size = 16
learning_rate = 1e-3
epochs = 3


if dataset == 'pv':
    train_dataset = train_dataset_pv
    val_dataset = val_dataset_pv
    test_dataset = test_dataset_pv
    n_features = train_dataset_pv.tensors[0].shape[-1]
else:
    train_dataset = train_dataset_forecast
    val_dataset = val_dataset_forecast
    test_dataset = test_dataset_forecast
    n_features = train_dataset.example[0].shape[-1]

train_loader = DataLoader(train_dataset, batch_size= batch_size, shuffle= True)
val_loader = DataLoader(val_dataset, batch_size= batch_size, shuffle= True)
test_loader = DataLoader(test_dataset, batch_size= batch_size, shuffle= True)

model = RNNModel(p = n_features)
loss_fn = nn.MSELoss()
optim = torch.optim.Adam(params= model.parameters(), lr= learning_rate)


def training_one_epoch(model, train_dataloader, val_dataloader):
    size = len(train_dataloader.dataset)
    running_loss = 0
    earlystopping = 0
    for batch, data in enumerate(train_dataloader):
        X, y = data
        output = model(X)
        loss = loss_fn(output, y)

        optim.zero_grad()
        loss.backward()
        optim.step()
        running_loss += loss.item()

        if batch % 10 == 9:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"\tloss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    with torch.no_grad():
        outputs = []
        labels = []
        vsize = len(val_dataloader.dataset)
        running_vloss = 0.0

        # In evaluation mode some model specific operations can be omitted eg. dropout layer
        model.train(False) # Switching to evaluation mode, eg. turning off regularisation
        for j, vdata in enumerate(val_dataloader):
            vinputs, vlabels = vdata
            voutputs = model(vinputs)
            vloss = loss_fn(voutputs, vlabels)
            running_vloss += vloss.item()

            if j % 10 == 9:
                vloss, vcurrent = vloss.item(), j * batch_size + len(vinputs)
                print(f"\tval loss: {vloss:>7f}  [{vcurrent:>5d}/{vsize:>5d}]")

        model.train(True) # Switching back to training mode, eg. turning on regularisation

        # avg_loss = running_loss / 1000
        # avg_vloss = running_vloss / len(val_loader_forecast)



for epoch in range(epochs):
    print(f'Epoch {epoch + 1}:')
    model.train()
    training_one_epoch(model, train_loader, val_loader)


# Example usage
metrics = compute_regression_metrics(model, test_loader)
print("Regression Metrics:", metrics)

If you start TensorBoard at the command line and open it in a new browser tab (usually at [localhost:6006](localhost:6006)), you should see the image grid under the IMAGES tab.

## Graphing Scalars to Visualize Training

TensorBoard is useful for tracking the progress and efficacy of your training. Below, we'll run a training loop, track some metrics, and save the data for TensorBoard's consumption.

Let's define a model to categorize our image tiles, and an optimizer and loss function for training:

In [ ]:
from torch.utils.tensorboard import SummaryWriter

# Default log_dir argument is "runs" - but it's good to be specific
# torch.utils.tensorboard.SummaryWriter is imported above
writer = SummaryWriter()
# writer.close()

# Write image data to TensorBoard log dir
# writer.add_image('Four Fashion-MNIST Images', img_grid)
# writer.flush()

# To view, start TensorBoard on the command line with:
#   tensorboard --logdir=runs
# ...and open a browser tab to http://localhost:6006/

In [ ]:
############# Data ##############
dataset = 'forecast'  # ['pv', 'forecast']

########## Parameters #########
batch_size = 32
learning_rate = 1e-3
epochs = 10


if dataset == 'pv':
    train_dataset = train_dataset_pv
    test_dataset = test_dataset_pv
    n_features = train_dataset_pv.tensors[0].shape[-1]
else:
    train_dataset = train_dataset_forecast
    test_dataset = test_dataset_forecast
    n_features = train_dataset.example[0].shape[-1]

train_loader = DataLoader(train_dataset, batch_size= batch_size, shuffle= True)
test_loader = DataLoader(test_dataset, batch_size= batch_size, shuffle= True)

model = LSTMModel(p = n_features)
loss_fn = nn.MSELoss()
optim = torch.optim.Adam(params= model.parameters(), lr= learning_rate)

print(len(val_loader_forecast))
for epoch in range(epochs):  # loop over the dataset multiple times
    running_loss = 0.0

    for i, data in enumerate(train_loader_forecast, 0):
        # basic training loop
        inputs, labels = data
        optim.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optim.step()

        running_loss += loss.item()
        if i % 8 == 7:    # Every 1000 mini-batches...
            print('Batch {}'.format(i + 1))
            # Check against the validation set
            running_vloss = 0.0

            # In evaluation mode some model specific operations can be omitted eg. dropout layer
            model.train(False) # Switching to evaluation mode, eg. turning off regularisation
            for j, vdata in enumerate(val_loader_forecast, 0):
                vinputs, vlabels = vdata
                voutputs = model(vinputs)
                vloss = loss_fn(voutputs, vlabels)
                running_vloss += vloss.item()
            model.train(True) # Switching back to training mode, eg. turning on regularisation

            avg_loss = running_loss / 1000
            avg_vloss = running_vloss / len(val_loader_forecast)

            # Log the running loss averaged per batch
            writer.add_scalars('Training vs. Validation Loss',
                            { 'Training' : avg_loss, 'Validation' : avg_vloss },
                            epoch * len(train_loader_forecast) + i)

            running_loss = 0.0
print('Finished Training')

writer.flush()

Switch to your open TensorBoard and have a look at the SCALARS tab.

## Visualizing Your Model
TensorBoard can also be used to examine the data flow within your model. To do this, call the `add_graph()` method with a model and sample input. When you open 

In [ ]:
# Again, grab a single mini-batch of images
dataiter = iter(train_loader_forecast)
inputs, labels = next(dataiter)

# add_graph() will trace the sample input through your model,
# and render it as a graph.
writer.add_graph(model, inputs)
writer.flush()
writer.close

In [ ]:
baseline_pv, baseline_forecast